# 2-15 Stochastic Methods, Part 2
* Simulated Annealing
* Genetic Algorithms

In [2]:
using Revealables
include("files/extras.jl")
include("files/answers.jl")

##Simulated Annealing
__Simulated Annealing__ is used for functions with many local extremes. Its goal is to return the global extreme as often as possible and, if not, then another almost-as-good extreme.

*Annealing* is a term used in metallurgy to describe a specific procedure in the refining of metals where the metal is heated and then slowly cooled.

This is the type of graph for which simulated annealing might be appropriate:
<img src="files/2-15/aplstock.png" width=400 alt="Apple's stock between summer 2012 and summer 2014" />

This graph has a ton of local minima and maxima. Testing points would very likely not produce a local maximum or minimum because the intervals would have to be very narrow to catch the true maximum. Sawtooth would work just as well for this particular graph, but there is no feasible version of Sawtooth for multi-variable situations. Simulated annealing can be used for any number of variables.

Simulated annealing begins with an initial point and tests neighboring points.

Unlike traditional optimization in which the neighboring point is immediately rejected if it is "worse" than the original, simulated annealing will accept some "worse" points in the beginning in an attempt to eventually find a global extreme.

Here is an example of simulated annealing in action (from Wikipedia contributer Kingpin13):

<img src="files/2-15/Hill_Climbing_with_Simulated_Annealing.gif" />

In the beginning the test line (red) is everywhere; towards the middle it's dancing mostly between the high points of the graph; nearer the end it has found the two highest points; at the very end it settles on the global maximum.

The "temperature" measure indicates the willingness of the program to accept "worse" answers: high at first, lower later.

It's called “temperature” because the term annealing is borrowed from metallurgy where it describes a process of refinement by heating and then slowly cooling materials.

In metallurgy, the slower the rate of cooling the better the refining process works, so ideally you would want the temperature to remain high for a while as it goes through a nice, slow decrease. The same thing happens here: the temperature should remain high in the beginning so the function gets to test a lot of points (being very willing to accept "worse" solutions); then as time goes by it will gradually become less willing to do so, eventually settling in on the optimum (or close to it). 

If the algorithm for simulated annealing is run for long enough, it will always find the global extreme. However, it can sometimes be prohibitive to run that many calculations.

One of the advantages of simulated annealing is that even if you stop it a little early, its current solution will either be the optimum or really close to it.

###Pseudocode for Simulated Annealing
Here is a section of *pseudocode* (again from Wikipedia) showing the simulated annealing procedure for minimization:

```
s ← s0; e ← E(s)                        // Initial state energy.
sbest ← s; ebest ← e                    // Initial "best" solution
k ← 0                                    // Energy evaluation count.
while k < kmax and e > emax              // While time left & not good enough:
    T ← temperature(k/kmax)              // Temperature calculation.
    snew ← neighbour(s)                  // Pick some neighbour.
    enew ← E(snew)                       // Compute its energy.
    if P(e enew T) > random() then        // Should we move to it?
        s ← snew; e ← enew              // Yes change state.
    if enew < ebest then                  // Is this a new best?
        sbest ← snew; ebest ← enew       // Save 'new neighbour' to 'best found'.
    k ← k + 1                            // One more evaluation done
return sbest                              // Return the best solution found.
```
This pseudocode (literally "fake code") is meant to be easily readable by people.

Notice:
* the random number generator in there. That's what makes this a stochastic process. 
* the heavy documentation which makes sense of everything even though it's written in pseudocode and not in our language.
* the iteration counter which allows us to limit the number of iterations at will.
* the temperature counter. As `k` increases `k/kmax` changes then `T` will change (lower according to the procedure) as calculated by the unspecified function `temperature(x)`. This `T` then gets plugged into `P` later. 
* `P` is also an unspecified function, but according to the procedure, `P` will control the shifting so that as temperature increases the probability of accepting a lower value will decrease. This `P` value will still be checked against a random number, so a lower `P` does not necessarily guarantee a lack of motion, just a lower probability of it.
* `e`, the "energy," is the objective function the goal being to minimize the energy of the state; again borrowed from metallurgy. The `e > emax` in the while loop allows an upper bound to acceptable returns such that even if the iterations run out if your energy is still higher than is acceptable the loop will continue to run.



###Practice Problem A
Translate as much as possible of the pseudocode below into Julia.

In [ ]:
# s ← s0; e ← E(s)                        // Initial state energy.
# sbest ← s; ebest ← e                    // Initial "best" solution
# k ← 0                                    // Energy evaluation count.
# while k < kmax and e > emax              // While time left & not good enough:
#     T ← temperature(k/kmax)              // Temperature calculation.
#     snew ← neighbour(s)                  // Pick some neighbour.
#     enew ← E(snew)                       // Compute its energy.
#     if P(e enew T) > random() then        // Should we move to it?
#         s ← snew; e ← enew              // Yes change state.
#     if enew < ebest then                  // Is this a new best?
#         sbest ← snew; ebest ← enew       // Save 'new neighbour' to 'best found'.
#     k ← k + 1                            // One more evaluation done
# return sbest                              // Return the best solution found.

In [3]:
revealable(ans215A)

Interact.ToggleButton(1: "input" = false Bool , "Answer", false)

  likely near C:\Users\Victoria Docherty\.julia\v0.6\IJulia\src\kernel.jl:31
  likely near C:\Users\Victoria Docherty\.julia\v0.6\IJulia\src\kernel.jl:31
  likely near C:\Users\Victoria Docherty\.julia\v0.6\IJulia\src\kernel.jl:31


##Genetic Algorithms
__Genetic algorithms__ attempt to find optimal solutions by using concepts related to the process of natural selection: 

* inheritance 
* mutation 
* selection, and
* crossover.

###Genetic Algorithms: Building a Population
Each "individual" is represented by a string, usually in binary; for example:

    1111100010100111011010001011011001011011
    
Fortunately any number can be converted into binary and back so this string of 0's and 1's can be translated into the values of numerical variables. These values can then be plugged into an objective function.

In the beginning, many, many of these strings are randomly chosen.

###Genetic Algorithms: Testing Fitness
The next step is to find the value of the objective function for each "individual." This is done by translating the binary strings into numbers and plugging them in. The objective function value gives a measure of the individual's "fitness."

If you are trying to minimize, the "fittest" individuals would have the lowest values of the objective function. 

This is the bottleneck step in genetic algorithms. Evaluating the objective function, which is super-complex because otherwise we wouldn't be using stochastic methods in the first place, can take a very long time. Evaluating it multiple times over a large population is terribly cumbersome. The temptation is therefore to reduce the population and/or the number of iterations, but the whole point of stochastic methods is that you need a large sample. 

###Binary Numbers: A Review
With binary numbers, each digit space represents a power of two. This number:

	  1 0 0 1 1

...contains 1 sixteen, 0 eights, 0 fours, 1 two and 1 one. That adds up to 16 + 2 + 1 = 19.

Similarly, 101 = ?

1010 = ?


In [4]:
revealable(binary215)

Interact.ToggleButton(4: "input-2" = false Bool , "Answers", false)

###Practice Problem B
Here are four randomly-generated individuals:<br clear="all" />
A: 011111010001<br clear="all" />
B: 001000101001<br clear="all" />
C: 101110001010<br clear="all" />
D: 101101000111<br clear="all" />

1. Split each “phenotype” into three variables of length 4. Convert each into a base-10 number, represented by `a`, `b`, and `c`.

2. For each individual, calculate its "fitness" according to the objective function `a + b – c`, with the goal of *maximizing*.


In [ ]:
# Convert the numbers

# Calculate fitness


In [5]:
revealable(ans215B)

Interact.ToggleButton(7: "input-3" = false Bool , "Answer", false)

###Genetic Algorithms: Selection
After fitness is determined, the individuals are ranked from most to least fit. All of the individuals may contribute to the next "generation," but with different probabilities based on their fitness. 

Individuals are chosen randomly, in pairs, with repeats allowed, until the number chosen is equal to the original population.


###Practice Problem C
In Problem B, the fitness in order from high to low was A, C, D, B.

Randomly choose pairs of individuals according to the following method:

1. Use the command rand(1:10). If the result is 1, 2, 3, 4 choose A; 5, 6, 7 choose C; 8, 9 choose D; 10 choose B.
2. Individuals may not “breed” with themselves; choose another number if this happens.

Use this method to create two pairs.


In [ ]:
# Generate two pairs here

In [6]:
revealable(ans215C)

Interact.ToggleButton(10: "input-4" = false Bool , "Answer", false)

###Genetic Algorithms: Crossover
In the next step, a random number $n$ is selected between 1 and one less than the maximum string length. Then, the first $n$ characters of each pair are switched.

If you had the pair:



<font color="#336699">101</font><font color="#993333">10</font> and<br clear="all" />
<font color="#33CC66">011</font><font color="#993333">01</font>

with $n = 3$

you would switch the first three characters of each, creating two "children,"
<font color="#33CC66">011</font><font color="#993333">10</font> and<br clear="all" />
<font color="#336699">101</font><font color="#993333">01</font>

There are a ton of variations on this method (like swapping more than 1 set of points, or switching order), but this is the basic way it's done.

The reason $n$ can’t equal the maximum string length is that then the "children" would be identical to the "parents." 




###Practice Problem D1
For each pair from Problem C, use `rand(1:11)` to determine how many digits to swap; then, swap those digits.

In [ ]:
# Do your swapping here

In [7]:
revealable(ans215D1)

Interact.ToggleButton(13: "input-5" = false Bool , "Answer", false)

###Genetic Algorithms: Mutation
The process of "mutation" is a random process that prevents the "children" from being too similar to the "parents," which would make it difficult to break out of a local optimum and find the global one. 

In mutation, a probability is selected; it can be a chosen number between 0.005 and 0.1, or equal to $1\over{k}$ where $k$ is the string length (the second method guarantees an average of 1 mutation per child).

Each character is given that probability of randomly switching value from 0 to 1 or 1 to 0.


###Practice Problem D2
Use the command `rand(12)` to generate 12 random numbers between 0 and 1.

If any of the numbers is less than $0.08\overline{3}$ (that's $1\over12$), flip the corresponding character(s) on the first child from Problem D1.

Repeat for all four children.

Calculate the fitness of all four children.

In [ ]:
# Calculate new generations here

In [8]:
revealable(ans215D2)

Interact.ToggleButton(16: "input-6" = false Bool , "Answer", false)

###Genetic Algorithms: Repeat and Repeat and...
Using computers, these procedures are repeated over large numbers of individuals over many generations. Eventually, the "population" will become more "fit," just as natural selection works on a biological population.

In more mathematical terms, this means the variables and fitness will gradually converge on the optimal solution.

###Practice Problem E
Repeat the genetic algorithm for two more generations: pair the children with weighted probabilities according to fitness, swap genes, mutate, evaluate; then repeat once more.

In [ ]:
# Keep evolving...

In [9]:
revealable(ans215E)

Interact.ToggleButton(19: "input-7" = false Bool , "Answer", false)

###Genetic Algorithms: Disadvantages
One disadvantage of genetic algorithms is that they require a lot of evaluations of the objective function, which can be costly and time-consuming.

Another is that, while they work well for simpler systems, they do not scale up well to very complex systems: the longer the string, the less successful the method tends to be.

Genetic algorithms also tend to get stuck in local extremes.

###Genetic Algorithms: Why Do We Use Them?
In spite of the difficulties mentioned above, for simple systems (fewer variables) with many potential inputs (large domains) and complicated, interdependent, or nonexistent equations and conditions, genetic algorithms are surprisingly good at finding optimal solutions.